In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
#%cd D:\StonyBrook\Study\Prob&Stats CSE544\Project

from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My Drive/Prob_stats_proj

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Prob_stats_proj


#Data Preprocessing

In [ ]:
data = pd.read_csv('7.csv')

In [ ]:
## converting date column to datetime data type ##
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
data

,Date,IA confirmed cumulative,ID confirmed cumulative,IA deaths cumulative,ID deaths cumulative,IA confirmed,ID confirmed,IA deaths,ID deaths
0,2020-01-22,0,0,0,0,0,0,0,0
1,2020-01-23,0,0,0,0,0,0,0,0
2,2020-01-24,0,0,0,0,0,0,0,0
3,2020-01-25,0,0,0,0,0,0,0,0
4,2020-01-26,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
433,2021-03-30,349742,179429,5726,1956,141,0,7,0
434,2021-03-31,350840,180536,5744,1962,1098,1107,18,6
435,2021-01-04,351651,180897,5744,1963,811,361,0,1
436,2021-02-04,352262,181181,5752,1966,611,284,8,3


###Data features
Following list displays the features in our data
 

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     438 non-null    datetime64[ns]
 1   IA confirmed cumulative  438 non-null    int64         
 2   ID confirmed cumulative  438 non-null    int64         
 3   IA deaths cumulative     438 non-null    int64         
 4   ID deaths cumulative     438 non-null    int64         
 5   IA confirmed             438 non-null    int64         
 6   ID confirmed             438 non-null    int64         
 7   IA deaths                438 non-null    int64         
 8   ID deaths                438 non-null    int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 30.9 KB


In [ ]:
def generate_data(p,data):
    X, Y = [], []
    for i in range(len(data) - p):
        X.append([1] + data[i:i+p].tolist())  # appending 1 for beta_0
    
    Y = data[p:].tolist()

    return np.array(X),np.array(Y)

In [ ]:
def SSE(y_pred, y_true):
    A = y_true - y_pred
    A = A*A
    return sum(A)

def MSE(y_pred,y_true):
    return SSE(y_pred,y_true)/len(y_pred)

def MAPE(y_pred, y_true):
    A = np.abs(y_true - y_pred)
    A = np.sum(np.divide(A, y_true))*100
    return A/len(y_pred)

In [ ]:
def AutoRegression(p, data):
    X, Y = generate_data(p, data)
    A = np.linalg.inv(np.matmul(np.transpose(X), X))
    B = np.matmul(np.transpose(X), Y)
    beta = np.matmul(A,B)
    
    return beta

def Predict_AR(data, beta):
    p = len(beta) - 1 
    data = data.to_numpy()
    for i in range(7):
        y_pred = beta[0] + np.matmul(data[-p:].T, beta[1:])
        data = np.append(data,y_pred)
    
    return data

In [ ]:
###### Outlier Detection Using Tukey's Rule ########
## Plot data ####

In [ ]:
###### Getting August 2020 data ######
start_date, end_date = '2020-08-01', '2020-08-29'
condition = (data['Date'] >= start_date) & (data['Date'] <= end_date)
august_data = data.loc[condition]
august_first_3_week_data = august_data[:-7]
print(august_first_3_week_data)


          Date  IA confirmed cumulative  ...  IA deaths  ID deaths
17  2020-08-02                        0  ...          0          0
46  2020-08-03                        0  ...          0          0
77  2020-08-04                     1145  ...          1          3
107 2020-08-05                    11447  ...         12          0
138 2020-08-06                    21907  ...          5          0
168 2020-08-07                    32388  ...          7          4
199 2020-08-08                    48112  ...         13          6
204 2020-08-13                    50167  ...          5          5
205 2020-08-14                    50808  ...         10         14
206 2020-08-15                    51640  ...          9          4
207 2020-08-16                    52306  ...          2          0
208 2020-08-17                    52617  ...          4          4
209 2020-08-18                    52930  ...          8          9
210 2020-08-19                    53538  ...         16       

# AutoRegression

##Autoregression(3)
The following displays the MSE and MAPE prediction scores for each of the following features using AutoRegression(3)

In [ ]:
##### AutoRegression(3) for IA confirmed cases #####
handle = 'IA confirmed'

beta_coefficients = AutoRegression(3,august_first_3_week_data[handle])
print('Beta Coefficients : ' , beta_coefficients)
predicted_values = Predict_AR(august_first_3_week_data[handle],beta_coefficients)[-7:]
print(predicted_values, august_data[handle][-7:])


print('\nMSE : ' , MSE(predicted_values, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(predicted_values, august_data[handle][-7:] ))


Beta Coefficients :  [ 4.01115518e+02  4.61310057e-02 -3.41190266e-02  2.81516663e-01]
[644.32650458 579.40024984 582.05360773 574.92816453 569.83659011
 568.76874233 568.31314108] 218    1477
219    2535
220    1081
230     327
260    1515
291    4249
321    1393
Name: IA confirmed, dtype: int64

MSE :  2849382.258072868

MAPE :  66.2426116733827


In [ ]:
##### AutoRegression(3) for IA deaths cases #####
handle = 'IA deaths'

beta_coefficients = AutoRegression(3,august_first_3_week_data[handle])
print('Beta Coefficients : ' , beta_coefficients)
predicted_values = Predict_AR(august_first_3_week_data[handle],beta_coefficients)[-7:]
print(predicted_values, august_data[handle][-7:])


print('\nMSE : ' , MSE(predicted_values, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(predicted_values, august_data[handle][-7:] ))


Beta Coefficients :  [11.53128227  0.09908033 -0.33632856 -0.19175839]
[6.50686783 6.80298747 9.32635117 8.09953984 7.51545155 8.29008278
 8.21643344] 218    18.0
219    11.0
220    17.0
230     6.0
260     5.0
291    14.0
321     5.0
Name: IA deaths, dtype: float64

MSE :  37.46800718875394

MAPE :  48.22280826487738


In [ ]:
##### AutoRegression(3) for ID confirmed cases #####
handle = 'ID confirmed'

beta_coefficients = AutoRegression(3,august_first_3_week_data[handle])
print('Beta Coefficients : ' , beta_coefficients)
predicted_values = Predict_AR(august_first_3_week_data[handle],beta_coefficients)[-7:]
print(predicted_values, august_data[handle][-7:])


print('\nMSE : ' , MSE(predicted_values, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(predicted_values, august_data[handle][-7:] ))


Beta Coefficients :  [ 2.88994331e+02  1.41280277e-01 -1.31951689e-01  9.77084580e-02]
[322.15395554 337.48205881 323.11602982 321.54817264 325.4561602
 324.01524849 323.13728637] 218     342.0
219     262.0
220     293.0
230     314.0
260     673.0
291     649.0
321    2013.0
Name: ID confirmed, dtype: float64

MSE :  441299.0224364499

MAPE :  33.27977484884011


In [ ]:
##### AutoRegression(3) for ID deaths cases #####
handle = 'ID deaths'

beta_coefficients = AutoRegression(3,august_first_3_week_data[handle])
print('Beta Coefficients : ' , beta_coefficients)
predicted_values = Predict_AR(august_first_3_week_data[handle],beta_coefficients)[-7:]
print(predicted_values, august_data[handle][-7:])


print('\nMSE : ' , MSE(predicted_values, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(predicted_values, august_data[handle][-7:] ))


Beta Coefficients :  [ 5.33182013  0.12268488 -0.16319995  0.09989717]
[5.45376871 5.55365534 6.34609351 5.72851563 5.54975006 5.72990074
 5.70130435] 218     6.0
219    10.0
220     5.0
230     4.0
260     3.0
291     3.0
321    19.0
Name: ID deaths, dtype: float64

MSE :  30.810996120189202

MAPE :  52.81193682036503


##Autoregression(5)
The following displays the MSE and MAPE prediction scores for each of the features using AutoRegression(5)

In [ ]:
##### AutoRegression(5) for IA confirmed cases #####
handle = 'IA confirmed'

beta_coefficients = AutoRegression(5,august_first_3_week_data[handle])
print('Beta Coefficients : ' , beta_coefficients)
predicted_values = Predict_AR(august_first_3_week_data[handle],beta_coefficients)[-7:]
print(predicted_values, august_data[handle][-7:])


print('\nMSE : ' , MSE(predicted_values, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(predicted_values, august_data[handle][-7:] ))

Beta Coefficients :  [ 5.14180039e+02  1.76266604e-01  7.34411976e-02 -1.95422993e-01
 -1.43606627e-01  2.33540465e-01]
[737.35148492 598.80745804 496.86423946 564.15777609 663.83002105
 665.04329834 605.95483127] 218    1477.0
219    2535.0
220    1081.0
230     327.0
260    1515.0
291    4249.0
321    1393.0
Name: IA confirmed, dtype: float64

MSE :  2697436.5647138814

MAPE :  64.29273957596156


In [ ]:
##### AutoRegression(5) for ID confirmed cases #####
handle = 'ID confirmed'

beta_coefficients = AutoRegression(5,august_first_3_week_data[handle])
print('Beta Coefficients : ' , beta_coefficients)
predicted_values = Predict_AR(august_first_3_week_data[handle],beta_coefficients)[-7:]
print(predicted_values, august_data[handle][-7:])


print('\nMSE : ' , MSE(predicted_values, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(predicted_values, august_data[handle][-7:] ))

Beta Coefficients :  [ 6.18167637e+02 -3.30124894e-01 -2.83777575e-02 -6.12995878e-02
 -2.75798717e-01 -1.89320707e-01]
[325.43553517 434.82637992 282.50288827 281.0205596  349.15229077
 337.46968869 289.19162733] 218     342.0
219     262.0
220     293.0
230     314.0
260     673.0
291     649.0
321    2013.0
Name: ID confirmed, dtype: float64

MSE :  457826.42160865356

MAPE :  38.09267433885999


In [ ]:
##### AutoRegression(5) for IA deaths cases #####
handle = 'IA deaths'

beta_coefficients = AutoRegression(5,august_first_3_week_data[handle])
print('Beta Coefficients : ' , beta_coefficients)
predicted_values = Predict_AR(august_first_3_week_data[handle],beta_coefficients)[-7:]
print(predicted_values, august_data[handle][-7:])


print('\nMSE : ' , MSE(predicted_values, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(predicted_values, august_data[handle][-7:] ))

Beta Coefficients :  [ 1.60707672e+01 -1.52382702e-02 -3.05734957e-01 -3.26461016e-02
 -5.66014091e-01 -7.70931090e-02]
[8.08442717 6.19064935 7.76540583 7.59248926 8.21821942 8.87034239
 8.01894097] 218    18.0
219    11.0
220    17.0
230     6.0
260     5.0
291    14.0
321     5.0
Name: IA deaths, dtype: float64

MSE :  36.43521681960978

MAPE :  48.72201918574164


In [ ]:
##### AutoRegression(5) for ID deaths cases #####
handle = 'ID deaths'

beta_coefficients = AutoRegression(5,august_first_3_week_data[handle])
print('Beta Coefficients : ' , beta_coefficients)
predicted_values = Predict_AR(august_first_3_week_data[handle],beta_coefficients)[-7:]
print(predicted_values, august_data[handle][-7:])


print('\nMSE : ' , MSE(predicted_values, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(predicted_values, august_data[handle][-7:] ))

Beta Coefficients :  [ 8.94702298  0.03234352 -0.02956219 -0.10792331 -0.32372396 -0.10179831]
[3.14385467 3.53444318 6.2863271  6.88654457 6.0923404  5.41624981
 5.60868145] 218     6.0
219    10.0
220     5.0
230     4.0
260     3.0
291     3.0
321    19.0
Name: ID deaths, dtype: float64

MSE :  36.382287565793945

MAPE :  66.32120636165934


#EWMA

In [ ]:
def EWMA(alpha, Y):
    y_hat = 0.0
    Y = Y.to_numpy()
    for i in range(len(Y)-1):
        y_hat = (1-alpha)*(y_hat + Y[i])
    
    y_hat+= Y[-1]
    y_hat = y_hat*alpha
    
    return y_hat

def Predict_EWMA(alpha, Y, y_hat):
    Y = Y.to_numpy()
    pred_values = [y_hat]
    for i in range(1,7):
        y_hat = alpha*Y[i] + (1-alpha)*y_hat
        pred_values.append(y_hat)
    
    return pred_values

##EWMA (alpha =0.5)
The following displays the MSE and MAPE prediction scores for each of the following features using EWMA, with alpha=0.5

In [ ]:
##### EWMA(0.5) for IA confirmed cases #####
handle = 'IA confirmed'
alpha = 0.5

y_hat_t = EWMA(alpha,august_first_3_week_data[handle])
y_pred = Predict_EWMA(alpha,august_data[handle][-7:], y_hat_t)
print(y_pred, august_data[handle][-7:] )

print('\nMSE : ' , MSE(y_pred, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(y_pred, august_data[handle][-7:] ))

[713.611837387085, 1624.3059186935425, 1352.6529593467712, 839.8264796733856, 1177.4132398366928, 2713.2066199183464, 2053.103309959173] 218    1477.0
219    2535.0
220    1081.0
230     327.0
260    1515.0
291    4249.0
321    1393.0
Name: IA confirmed, dtype: float64

MSE :  665324.8616970623

MAPE :  53.626037631192894


In [ ]:
##### EWMA(0.5) for IA deaths cases #####
handle = 'IA deaths'
alpha = 0.5

y_hat_t = EWMA(alpha,august_first_3_week_data[handle])
y_pred = Predict_EWMA(alpha,august_data[handle][-7:], y_hat_t)
print(y_pred, august_data[handle][-7:] )

print('\nMSE : ' , MSE(y_pred, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(y_pred, august_data[handle][-7:] ))

[10.329507827758789, 10.664753913879395, 13.832376956939697, 9.916188478469849, 7.458094239234924, 10.729047119617462, 7.864523559808731] 218    18.0
219    11.0
220    17.0
230     6.0
260     5.0
291    14.0
321     5.0
Name: IA deaths, dtype: float64

MSE :  15.6094376148345

MAPE :  37.05438955974141


In [ ]:
##### EWMA(0.5) for ID confirmed cases #####
handle = 'ID confirmed'
alpha = 0.5

y_hat_t = EWMA(alpha,august_first_3_week_data[handle])
y_pred = Predict_EWMA(alpha,august_data[handle][-7:], y_hat_t)
print(y_pred, august_data[handle][-7:] )

print('\nMSE : ' , MSE(y_pred, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(y_pred, august_data[handle][-7:] ))

[329.5948257446289, 295.79741287231445, 294.3987064361572, 304.1993532180786, 488.5996766090393, 568.7998383045197, 1290.8999191522598] 218     342.0
219     262.0
220     293.0
230     314.0
260     673.0
291     649.0
321    2013.0
Name: ID confirmed, dtype: float64

MSE :  80465.46206931304

MAPE :  13.679243580916213


In [ ]:
##### EWMA(0.5) for ID deaths cases #####
handle = 'ID deaths'
alpha = 0.5

y_hat_t = EWMA(alpha,august_first_3_week_data[handle])
y_pred = Predict_EWMA(alpha,august_data[handle][-7:], y_hat_t)
print(y_pred, august_data[handle][-7:] )

print('\nMSE : ' , MSE(y_pred, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(y_pred, august_data[handle][-7:] ))

[9.825506210327148, 9.912753105163574, 7.456376552581787, 5.7281882762908936, 4.364094138145447, 3.6820470690727234, 11.341023534536362] 218     6.0
219    10.0
220     5.0
230     4.0
260     3.0
291     3.0
321    19.0
Name: ID deaths, dtype: float64

MSE :  12.092627398996058

MAPE :  37.92546472901987


##EWMA (alpha =0.8)
The following displays the MSE and MAPE prediction scores for each of the following features using EWMA, with alpha=0.8

In [ ]:
##### EWMA(0.8) for IA confirmed cases #####
handle = 'IA confirmed'
alpha = 0.8

y_hat_t = EWMA(alpha,august_first_3_week_data[handle])
y_pred = Predict_EWMA(alpha,august_data[handle][-7:], y_hat_t)
print(y_pred, august_data[handle][-7:] )

print('\nMSE : ' , MSE(y_pred, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(y_pred, august_data[handle][-7:] ))

[800.9405945894398, 2188.188118917888, 1302.4376237835777, 522.0875247567155, 1316.417504951343, 3662.483500990269, 1846.8967001980536] 218    1477.0
219    2535.0
220    1081.0
230     327.0
260    1515.0
291    4249.0
321    1393.0
Name: IA confirmed, dtype: float64

MSE :  179126.76992106796

MAPE :  28.44188981460253


In [ ]:
##### EWMA(0.8) for IA deaths cases #####
handle = 'IA deaths'
alpha = 0.8

y_hat_t = EWMA(alpha,august_first_3_week_data[handle])
y_pred = Predict_EWMA(alpha,august_data[handle][-7:], y_hat_t)
print(y_pred, august_data[handle][-7:] )

print('\nMSE : ' , MSE(y_pred, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(y_pred, august_data[handle][-7:] ))

[12.050563011667244, 11.21011260233345, 15.84202252046669, 7.968404504093338, 5.593680900818667, 12.318736180163734, 6.463747236032747] 218    18.0
219    11.0
220    17.0
230     6.0
260     5.0
291    14.0
321     5.0
Name: IA deaths, dtype: float64

MSE :  6.568162419074762

MAPE :  18.24835803837691


In [ ]:
##### EWMA(0.8) for ID confirmed cases #####
handle = 'ID confirmed'
alpha = 0.8

y_hat_t = EWMA(alpha,august_first_3_week_data[handle])
y_pred = Predict_EWMA(alpha,august_data[handle][-7:], y_hat_t)
print(y_pred, august_data[handle][-7:] )

print('\nMSE : ' , MSE(y_pred, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(y_pred, august_data[handle][-7:] ))

[326.08733287314243, 274.8174665746285, 289.3634933149257, 309.07269866298515, 600.214539732597, 639.2429079465194, 1738.248581589304] 218     342.0
219     262.0
220     293.0
230     314.0
260     673.0
291     649.0
321    2013.0
Name: ID confirmed, dtype: float64

MSE :  11619.466984883626

MAPE :  5.474665068110251


In [ ]:
##### EWMA(0.8) for ID deaths cases #####
handle = 'ID deaths'
alpha = 0.8

y_hat_t = EWMA(alpha,august_first_3_week_data[handle])
y_pred = Predict_EWMA(alpha,august_data[handle][-7:], y_hat_t)
print(y_pred, august_data[handle][-7:] )

print('\nMSE : ' , MSE(y_pred, august_data[handle][-7:] ))
print('\nMAPE : ', MAPE(y_pred, august_data[handle][-7:] ))

[10.98200034597992, 10.196400069195983, 6.039280013839196, 4.407856002767839, 3.281571200553568, 3.056314240110714, 15.811262848022144] 218     6.0
219    10.0
220     5.0
230     4.0
260     3.0
291     3.0
321    19.0
Name: ID deaths, dtype: float64

MSE :  5.19369259424398

MAPE :  20.575002182202958
